In [103]:
import websocket
import _thread
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [104]:
image = cv2.imread('test_image/savedImage.jpg')
print(image.shape)
# cv2.imshow('result',image)
# cv2.waitKey(0)
plt.imshow(image)
plt.show()
img_height = image.shape[0]
img_width = image.shape[1]
lane_image = np.copy(image)

(120, 160, 3)


In [105]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)
    
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(image, vertices):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, vertices, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [106]:
# 'Draw_Line' module
def line_classify(lines):
    """
    :params: 
        lines: straight lines obtained through the hough_lines function
    :output: 
        pos_slope_pt: line set with positive slope (threshold: slope > 0.5)
        neg_slope_pt: line set with negative slope (threshold: slope < -0.5)
    """
    pos_slope_pt = []
    neg_slope_pt = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y1 - y2) / (x1 - x2 + 0.00001)
            if slope > 0.5:
                pos_slope_pt.append([x1, y1])
                pos_slope_pt.append([x2, y2])
            elif slope < -0.5:
                neg_slope_pt.append([x1, y1])
                neg_slope_pt.append([x2, y2])
               
    return pos_slope_pt, neg_slope_pt


def find_endpoints(point_sets, y_top, y_bottom):
    """
    :params: 
        point_sets: a set of points with format of [[x1, y1], [x2, y2]...]
        y_top: Upper bound of the y coordinate of the end point of the line
        y_bottom: Lower bound of the y coordinate of the end point of the line
    :output: 
        point1, point2: A pair of endpoints of the line after polyfit
    """
    if point_sets:      
        
        # find positive slope line
        point_sets = np.array(point_sets)
        pos_slope_x = point_sets[:, 0]
        pos_slope_y = point_sets[:, 1]
        
        pos_line = np.polyfit(pos_slope_x, pos_slope_y, 1)
        pos_slope = pos_line[0]
        pos_intercept = pos_line[1]
        
        # draw positive slope line
        posx_top = int((y_top - pos_intercept) / (pos_slope + 0.0001))
        posx_bottom = int((y_bottom - pos_intercept) / (pos_slope + 0.0001))
        point1 = (posx_top, y_top)
        point2 = (posx_bottom, y_bottom)
        endpoints_pair = (point1, point2)
        return endpoints_pair

def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    pos_slope_pt = []
    neg_slope_pt = []
    
    pos_slope_pt, neg_slope_pt = line_classify(lines)

    y_top = img.shape[0]
    y_bottom = int(0.4 * img.shape[0])
    
    pos_endpoints = find_endpoints(pos_slope_pt, y_top, y_bottom)
    # continue the next draw_line function only when endpoints are not empty.
    if pos_endpoints:
        point1 = pos_endpoints[0]
        point2 = pos_endpoints[1]
        cv2.line(img, point1, point2, color, thickness)
    
    neg_endpoints = find_endpoints(neg_slope_pt, y_top, y_bottom)
    if neg_endpoints:    
        point3 = neg_endpoints[0]
        point4 = neg_endpoints[1]
        cv2.line(img, point3, point4, color, thickness)

In [107]:
def lane_finding(image):
    # Read in and grayscale the image
    # image = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')
    gray = grayscale(image)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 7
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 100
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using region_of_interest
    height = image.shape[0]
    width = image.shape[1]
    vertices = np.array([[(0, 0.8*height), 
                        (0, 0.95*height), 
                        (width, 0.95*height),
                        (width, 0.8*height),
                        (0.5*width, 0.4*height)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
         
    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    lines_edges = weighted_img(color_edges, line_image, α=0.8, β=1., γ=0.)
    
    # Draw the image with lines on it
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_lines = weighted_img(image, line_image, α=0.8, β=1., γ=0.)
    
    return lines_edges, image_lines

In [108]:
plt.figure(figsize=(6, 8), dpi=100)
lines_edges, image_lines = lane_finding(lane_image)
plt.subplot(2, 1, 1)
plt.imshow(lines_edges)
plt.subplot(2, 1, 2)
plt.imshow(image_lines)